In [41]:
import sys
import pickle
import matplotlib.pyplot
import pandas as pd
import numpy as np
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
from sklearn.metrics import accuracy_score , precision_score, recall_score , f1_score

In [93]:
### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
features_list_finance = ['poi','salary', 'bonus', 'long_term_incentive', 'deferred_income', 'deferral_payments', 'loan_advances','other',
                         'expenses','director_fees','total_payments','exercised_stock_options', 'restricted_stock',
                         'restricted_stock_deferred', 'total_stock_value']
features_list_email = ['poi', 'to_messages', 'from_messages','from_this_person_to_poi', 
                         'from_poi_to_this_person','shared_receipt_with_poi' ]
features_list = ['poi','salary', 'bonus', 'long_term_incentive', 'deferred_income', 'deferral_payments', 'loan_advances','other',
                         'expenses','director_fees','total_payments','exercised_stock_options', 'restricted_stock',
                         'restricted_stock_deferred', 'total_stock_value','to_messages', 'from_messages','from_this_person_to_poi', 
                         'from_poi_to_this_person','shared_receipt_with_poi'] # You will need to use more features
### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

In [94]:
#Transform data to dataframe
data_dict_keys = pd.DataFrame(data_dict.keys(), columns=['person'])
data_dict_values = pd.DataFrame(data_dict.values())
df_enron = pd.concat([data_dict_keys, data_dict_values], axis = 1)

In [95]:
#Remove Outliers
df_enron.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 22 columns):
person                       146 non-null object
bonus                        146 non-null object
deferral_payments            146 non-null object
deferred_income              146 non-null object
director_fees                146 non-null object
email_address                146 non-null object
exercised_stock_options      146 non-null object
expenses                     146 non-null object
from_messages                146 non-null object
from_poi_to_this_person      146 non-null object
from_this_person_to_poi      146 non-null object
loan_advances                146 non-null object
long_term_incentive          146 non-null object
other                        146 non-null object
poi                          146 non-null bool
restricted_stock             146 non-null object
restricted_stock_deferred    146 non-null object
salary                       146 non-null object
shared_receipt_

In [96]:
for i in features_list_finance:
    df_enron[i].replace('NaN', 0, inplace=True)

In [97]:
for i in features_list_email:
    poi_median = df_enron[df_enron['poi'] == True][i].median()
    non_poi_median = df_enron[df_enron['poi'] == False][i].median()
    df_poi = df_enron[df_enron['poi'] == True]
    df_non_poi = df_enron[df_enron['poi'] == False]
    df_poi[i].replace('NaN', poi_median, inplace=True)
    df_non_poi[i].replace('NaN', non_poi_median, inplace=True)
df_enron = df_poi.append(df_non_poi)    

In [98]:
df_enron.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,total_payments,total_stock_value
count,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,146.000000,1.460000e+02,1.460000e+02
mean,1.333474e+06,4.387965e+05,-3.827622e+05,1.942249e+04,4.182736e+06,7.074827e+04,1.149658e+06,6.646839e+05,5.854318e+05,1.749257e+06,2.051637e+04,3.658114e+05,964.356164,4.350622e+06,5.846018e+06
std,8.094029e+06,2.741325e+06,2.378250e+06,1.190543e+05,2.607040e+07,4.327163e+05,9.649342e+06,4.046072e+06,3.682345e+06,1.089995e+07,1.439661e+06,2.203575e+06,950.962501,2.693448e+07,3.624681e+07
min,0.000000e+00,-1.025000e+05,-2.799289e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.604490e+06,-7.576788e+06,0.000000e+00,2.000000,0.000000e+00,-4.409300e+04
25%,0.000000e+00,0.000000e+00,-3.792600e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.115000e+03,0.000000e+00,0.000000e+00,590.250000,9.394475e+04,2.288695e+05
50%,3.000000e+05,0.000000e+00,0.000000e+00,0.000000e+00,6.082935e+05,2.018200e+04,0.000000e+00,0.000000e+00,9.595000e+02,3.605280e+05,0.000000e+00,2.105960e+05,594.000000,9.413595e+05,9.659550e+05
75%,8.000000e+05,9.684500e+03,0.000000e+00,0.000000e+00,1.714221e+06,5.374075e+04,0.000000e+00,3.750648e+05,1.506065e+05,8.145280e+05,0.000000e+00,2.708505e+05,1117.500000,1.968287e+06,2.319991e+06
max,9.734362e+07,3.208340e+07,0.000000e+00,1.398517e+06,3.117640e+08,5.235198e+06,8.392500e+07,4.852193e+07,4.266759e+07,1.303223e+08,1.545629e+07,2.670423e+07,5521.000000,3.098866e+08,4.345095e+08


In [99]:
data_dict.pop('TOTAL', 0)
data_dict.pop('THE TRAVEL AGENCY IN THE PARK', 0)

{'bonus': 'NaN',
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'NaN',
 'exercised_stock_options': 'NaN',
 'expenses': 'NaN',
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 362096,
 'poi': False,
 'restricted_stock': 'NaN',
 'restricted_stock_deferred': 'NaN',
 'salary': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 362096,
 'total_stock_value': 'NaN'}

In [100]:
### Task 3: Create new feature(s)
def computeFraction( poi_messages, all_messages ):
    """ given a number messages to/from POI (numerator) 
        and number of all messages to/from a person (denominator),
        return the fraction of messages to/from that person
        that are from/to a POI
   """


    ### you fill in this code, so that it returns either
    ###     the fraction of all messages to this person that come from POIs
    ###     or
    ###     the fraction of all messages from this person that are sent to POIs
    ### the same code can be used to compute either quantity

    ### beware of "NaN" when there is no known email address (and so
    ### no filled email features), and integer division!
    ### in case of poi_messages or all_messages having "NaN" value, return 0.
    fraction = []
    for i in poi_messages.keys():
        if (poi_messages[i] == "NaN"):
            fraction.append(0)
        elif (all_messages[i] == "NaN"):
            fraction.append(0)
        else:
            fraction.append(float(poi_messages[i])/float(all_messages[i]))


    return fraction

In [101]:
from_poi_to_this_person = df_enron["from_poi_to_this_person"]
to_messages = df_enron["to_messages"]
fraction_from_poi = computeFraction( from_poi_to_this_person, to_messages )
df_enron["fraction_from_poi"] = fraction_from_poi


from_this_person_to_poi = df_enron["from_this_person_to_poi"]
from_messages = df_enron["from_messages"]
fraction_to_poi = computeFraction( from_this_person_to_poi, from_messages )
df_enron["fraction_to_poi"] = fraction_to_poi

In [102]:
features_list.append('fraction_from_poi')
features_list.append('fraction_to_poi')

In [112]:
data_dict_mod = df_enron.set_index(keys='person')
data_dict_mod = data_dict_mod.to_dict(orient='index')

In [113]:
### Store to my_dataset for easy export below.
my_dataset = data_dict_mod

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

In [114]:
### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

# Provided to give you a starting point. Try a variety of classifiers.
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(features, labels)
pred = clf.predict(features)

print ("Accuracy score: " , accuracy_score(labels, pred))
print ("Recall score: " , recall_score(labels, pred))
print ("Precision score: " , precision_score(labels, pred))
print ("f1 score: " , f1_score(labels, pred))

('Accuracy score: ', 0.3424657534246575)
('Recall score: ', 1.0)
('Precision score: ', 0.15789473684210525)
('f1 score: ', 0.2727272727272727)


In [115]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(features, labels)
pred = clf.predict(features)

print ("Accuracy score: " , accuracy_score(pred, labels))
print ("Recall score: " , recall_score(pred, labels))
print ("Precision score: " , precision_score(pred, labels))
print ("f1 score: " , f1_score(pred, labels))

('Accuracy score: ', 1.0)
('Recall score: ', 1.0)
('Precision score: ', 1.0)
('f1 score: ', 1.0)


In [116]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(features, labels)
pred = clf.predict(features)

print ("Accuracy score: " , accuracy_score(pred, labels))
print ("Recall score: " , recall_score(pred, labels))
print ("Precision score: " , precision_score(pred, labels))
print ("f1 score: " , f1_score(pred, labels))

('Accuracy score: ', 0.8972602739726028)
('Recall score: ', 0.6666666666666666)
('Precision score: ', 0.3333333333333333)
('f1 score: ', 0.4444444444444444)


In [117]:
### Task 5: Tune your classifier to achieve better than .3 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = \r
    train_test_split(features, labels, test_size=0.3, random_state=42)

In [125]:
#Selection of best 20% features
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile=20)
selector.fit(features_train, labels_train)
features_train_transformed = selector.transform(features_train)
features_test_transformed  = selector.transform(features_test)

In [126]:
clf = GaussianNB()
clf.fit(features_train_transformed, labels_train)
pred = clf.predict(features_test_transformed)

print ("Accuracy score: " , accuracy_score(pred, labels_test))
print ("Recall score: " , recall_score(pred, labels_test))
print ("Precision score: " , precision_score(pred, labels_test))
print ("f1 score: " , f1_score(pred, labels_test))

('Accuracy score: ', 0.8863636363636364)
('Recall score: ', 0.0)
('Precision score: ', 0.0)
('f1 score: ', 0.0)


In [128]:
clf = DecisionTreeClassifier()
clf.fit(features_train_transformed, labels_train)
pred = clf.predict(features_test_transformed)

print ("Accuracy score: " , accuracy_score(pred, labels_test))
print ("Recall score: " , recall_score(pred, labels_test))
print ("Precision score: " , precision_score(pred, labels_test))
print ("f1 score: " , f1_score(pred, labels_test))

('Accuracy score: ', 0.7954545454545454)
('Recall score: ', 0.2)
('Precision score: ', 0.6666666666666666)
('f1 score: ', 0.30769230769230765)


In [129]:
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(features_train_transformed, labels_train)
pred = clf.predict(features_test_transformed)

print ("Accuracy score: " , accuracy_score(pred, labels_test))
print ("Recall score: " , recall_score(pred, labels_test))
print ("Precision score: " , precision_score(pred, labels_test))
print ("f1 score: " , f1_score(pred, labels_test))

('Accuracy score: ', 0.8636363636363636)
('Recall score: ', 0.2)
('Precision score: ', 0.3333333333333333)
('f1 score: ', 0.25)


In [130]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)